In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import os

In [16]:
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
# options = Options()
# options.add_extension('C:/Users/dloso/AppData/Roaming/Mozilla/Firefox/Profiles/mrrn081x.default-release-1735758974795/extensions/uBlock0@raymondhill.net.xpi')
busiest_airports = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX', 'JFK', 'CLT', 'LAS', 'MCO',
                    'MIA', 'PHX', 'SEA', 'SFO', 'EWR', 'IAH', 'BOS', 'MSP', 'FLL',
                    'LGA', 'DTW', 'PHL', 'SLC', 'BWI', 'IAD', 'SAN', 'DCA', 'TPA',
                    'BNA', 'AUS', 'HNL', 'MDW', 'DAL', 'PDX']
month_days = {'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30,
                              'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dec': 31}
for airport in busiest_airports:
    df = pd.DataFrame()
    
    for cur_year in range(2021, 2026):
        if cur_year == 2020 or cur_year == 2024:
            month_days['Feb'] = 29
        else:
            month_days['Feb'] = 28
            
        for cur_month in range(1, 13):
            url = 'https://www.wunderground.com/history/monthly/' + 'K' + airport + '/date/' + str(cur_year) + '-' + str(cur_month)
            service = Service('C:/Users/dloso/Downloads/geckodriver-v0.36.0-win64/geckodriver.exe')
            driver = webdriver.Firefox(service=service)
            driver.install_addon('C:/Users/dloso/AppData/Roaming/Mozilla/Firefox/Profiles/mrrn081x.default-release-1735758974795/extensions/uBlock0@raymondhill.net.xpi')
            driver.get(url)
            
            try:
                # 1. Wait for the specific observation table container to load
                wait = WebDriverWait(driver, 2)
                table_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "observation-table")))
            
                # 2. Extract headers (Top Level and Subheaders)
                # WU often uses 'thead' or specific 'tr' classes for these
                header_rows = table_container.find_elements(By.TAG_NAME, "tr")[:2]
                
                # Extract all data rows
                data_rows = table_container.find_elements(By.TAG_NAME, "tr")[2:]

                # String out data into a list of lists
                all_data = []
                for row in data_rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    all_data.append([c.text for c in cols])
            
                data_dict = {}
                days_in_month = month_days[list(month_days.keys())[cur_month-1]]
                data_dict[all_data[0][0]] = [date[0] for date in all_data[1:1+days_in_month]]
                data_dict['Temp_' + all_data[1+days_in_month][0]] = [temp[0] for temp in all_data[2+days_in_month:2+days_in_month*2]]
                data_dict['Temp_' + all_data[1+days_in_month][1]] = [temp[1] for temp in all_data[2+days_in_month:2+days_in_month*2]]
                data_dict['Temp_' + all_data[1+days_in_month][2]] = [temp[2] for temp in all_data[2+days_in_month:2+days_in_month*2]]
                data_dict['WindSpeed_' + all_data[4+days_in_month*4][0]] = [windspeed[0] for windspeed in all_data[5+days_in_month*4:5+days_in_month*5]]
                data_dict['WindSpeed_' + all_data[4+days_in_month*4][1]] = [windspeed[1] for windspeed in all_data[5+days_in_month*4:5+days_in_month*5]]
                data_dict['WindSpeed_' + all_data[4+days_in_month*4][2]] = [windspeed[2] for windspeed in all_data[5+days_in_month*4:5+days_in_month*5]]
                data_dict['Pressure_' + all_data[5+days_in_month*5][0]] = [pressure[0] for pressure in all_data[6+days_in_month*5:6+days_in_month*6]]
                data_dict['Pressure_' + all_data[5+days_in_month*5][1]] = [pressure[1] for pressure in all_data[6+days_in_month*5:6+days_in_month*6]]
                data_dict['Pressure_' + all_data[5+days_in_month*5][2]] = [pressure[2] for pressure in all_data[6+days_in_month*5:6+days_in_month*6]]
                data_dict['Precipitation' + all_data[6+days_in_month*6][0]] = [precip[0] for precip in all_data[7+days_in_month*6:7+days_in_month*7]]

                df_temp = pd.DataFrame(data_dict)
                df_temp['Month'] = all_data[0][0]
                df_temp['Day'] = df_temp[list(month_days.keys())[cur_month-1]]
                df_temp['Year'] = cur_year
                df_temp['Date'] = pd.to_datetime(df_temp['Year'].astype(str) + '-' + df_temp['Month'] + '-' + df_temp['Day'].astype(str), format='%Y-%b-%d')
                df_temp.drop(columns=[list(month_days.keys())[cur_month-1], 'Month', 'Day', 'Year'], inplace=True)
                df = pd.concat([df, df_temp])
            finally:
                driver.quit()
                print(f'{airport} {list(month_days.keys())[cur_month-1]} {cur_year} complete.')

    df.to_csv('data/weather/' + airport + '_Weather.csv')
    print('Wrote data to data/weather/' + airport + '_Weather.csv')

ATL Jan 2021 complete.
ATL Feb 2021 complete.
ATL Jan 2022 complete.
ATL Feb 2022 complete.
ATL Jan 2023 complete.
ATL Feb 2023 complete.
ATL Jan 2024 complete.
ATL Feb 2024 complete.
ATL Jan 2025 complete.
ATL Feb 2025 complete.
Wrote data to data/weather/ATL_Weather.csv
DFW Jan 2021 complete.
DFW Feb 2021 complete.
DFW Jan 2022 complete.


KeyboardInterrupt: 